In [1]:
import sys
import pathlib

In [2]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter
from tqdm.auto import tqdm

In [3]:
%load_ext autoreload
%autoreload 2

In [13]:
sys.path.append("..")
from src.preprocess import preprocess, discrete_timestep
from src.utils import WorldDefinition, assign_color, point_to_bbox
from src.visualize import visualize_objects, ObjectVisualization

In [71]:
FREQUENCY = 30
DURATION = 3

In [72]:
data_path = "../data/raw/rdb1.parquet"
df = pd.read_parquet(data_path)

In [ ]:
df = df[df["table"] == "rdb1_1"]

In [ ]:
df["k"] = discrete_timestep(df, FREQUENCY)

In [ ]:
df = df[df["k"] < FREQUENCY*DURATION]

In [ ]:
df = df.sort_values("k")

In [ ]:
gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df["UTM_X"], df["UTM_Y"]), crs='EPSG:25832')

In [ ]:
gdf["bbox"] = gdf.apply(lambda row:
                            point_to_bbox(row.geometry, row["WIDTH"],
                                          row["LENGTH"],
                                          row["UTM_ANGLE"],
                                          as_linestring=True), axis=1)

In [ ]:
gdf = gdf.set_geometry("bbox")

In [ ]:
gdf.head()

In [ ]:
# get traffic lanes
trafficlanes_shapefile = pathlib.Path("../data/raw/rdb1/map_rdb1/shapefiles_trafficlanes")
trafficlanes = gpd.read_file(trafficlanes_shapefile)

In [ ]:
world_def = WorldDefinition.from_pgw_file(pathlib.Path(r'..\data\raw\rdb1\geo-referenced_images_rdb1\rdb1.pgw'))
extent = world_def.get_image_extent(3840, 2160)

In [ ]:
gdf["color"] = assign_color(gdf, column="OBJID")

In [ ]:
gdf.head()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 9), facecolor="w")

moviewriter = FFMpegWriter(FREQUENCY)

with moviewriter.saving(fig, 'test.gif', dpi=200):
    
    for k, timestep_data in tqdm(gdf.groupby("k"), total=gdf["k"].nunique()):
        ax.clear()
        
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        objects = [ObjectVisualization(i, row["bbox"], color=row["color"]) for i, row in timestep_data.iterrows()]

        visualize_objects(ax, objects, trafficlanes)

        ax.set_xlim(extent[0], extent[1])
        ax.set_ylim(extent[2], extent[3])

        moviewriter.grab_frame()